In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
import sklearn.linear_model

import warnings
warnings.filterwarnings("ignore")

from acquire_telco import get_telco_data
from prep_telco import prep_telco_data, split_telco_data, prep_telco_data_only

In [2]:
import dtale

# Gathering Data

In [3]:
# acquiring Telco data
df = get_telco_data()

In [ ]:
dtale.show(df)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
plt.figure(figsize = (8,4))
df.churn.hist(bins=3, align='mid')
plt.title('Has the customer churned?')
plt.ylabel('# of customers')
plt.show()


In [ ]:
df.monthly_charges.hist()
plt.title('What customers are paying each month')
plt.ylabel('# of customers')
plt.xlabel('Monthly Charge, $USD')
plt.show()

In [ ]:
df.contract_type.hist()
plt.title('customer contracts by type')
plt.ylabel('# of customers')
plt.xlabel('contract type')
plt.show()

In [ ]:
#dfc = prep_telco_data_only(df)
#dfc.head()

In [ ]:
#dfc = dfc.select_dtypes(exclude=['object'])

In [ ]:
#dtale.show(dfc)

In [ ]:
dtale.show(train)

In [4]:
# preperation and split of data
train, validate, test = prep_telco_data(df)

In [ ]:
train.describe()

In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
train.info()

In [5]:
# drop object dtypes
train = train.select_dtypes(exclude=['object'])
validate = validate.select_dtypes(exclude=['object'])
test = test.select_dtypes(exclude=['object'])

In [ ]:
train.info()


In [ ]:
dtale.show(train)

In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
train.churn_encoded.value_counts()

In [ ]:
train.info()

In [6]:
X_train = train.drop(columns=["churn_encoded"])
y_train = train.churn_encoded

X_validate = validate.drop(columns=["churn_encoded"])
y_validate = validate.churn_encoded

X_test = test.drop(columns=["churn_encoded"])
y_test = test.churn_encoded

# Establish target features
features = ['monthly_charges', 'internet_service_type_Fiber optic', 'internet_service_type_None', 'senior_citizen']
features2 = ['tenure','monthly_charges', 'partner_encoded','senior_citizen']
features3 = ['tenure','monthly_charges', 'partner_encoded','total_charges','senior_citizen']
features4 = ['tenure','monthly_charges', 'partner_encoded']
features5 = ['internet_service_type_Fiber optic', 'internet_service_type_None','tenure','monthly_charges']
features6 = ['tenure','contract_type_One year','contract_type_Two year','internet_service_type_Fiber optic','internet_service_type_None','monthly_charges']

In [ ]:
y_train[0:10]

In [ ]:
train.churn_encoded.value_counts()

In [7]:
# Create Baseline
baseline = y_train.mode()

# Produce a boolean array with True representing a match between the baseline prediction and reality
matches_baseline_prediction = (y_train == 0)

baseline_accuracy = matches_baseline_prediction.mean()
print(f"Baseline accuracy: {round(baseline_accuracy, 2)}")

Baseline accuracy: 0.73


# KNN - Train


In [ ]:
# Create knn model
knn = KNeighborsClassifier(n_neighbors = 5)
# Fit model to predictors and target
knn = knn.fit(X_train[features], y_train)
# Predict outcomes
y_pred = knn.predict(X_train[features])
# Probability estimates
y_pred_proba = knn.predict_proba(X_train[features])

# Print scoring
print(classification_report(y_train, y_pred))

In [ ]:
# feature model 1 is good and validation moddel stays in line with train


# Predict outcomes
y_pred = knn.predict(X_validate[features])
# Probability estimates
y_pred_proba = knn.predict_proba(X_validate[features])

# Print scoring
print(classification_report(y_validate, y_pred))

# knn feature 2

In [ ]:
# Create knn model
knn = KNeighborsClassifier(n_neighbors = 5)
# Fit model to predictors and target
knn = knn.fit(X_train[features2], y_train)
# Predict outcomes
y_pred = knn.predict(X_train[features2])
# Probability estimates
y_pred_proba = knn.predict_proba(X_train[features2])

# Print scoring
print(classification_report(y_train, y_pred))

In [ ]:
# features 2 performed better than feature 1

# Predict outcomes
y_pred = knn.predict(X_validate[features2])
# Probability estimates
y_pred_proba = knn.predict_proba(X_validate[features2])

# Print scoring
print(classification_report(y_validate, y_pred))

# knn feature 3

In [ ]:
# Create knn model
knn = KNeighborsClassifier(n_neighbors = 3)
# Fit model to predictors and target
knn = knn.fit(X_train[features3], y_train)
# Predict outcomes
y_pred = knn.predict(X_train[features3])
# Probability estimates
y_pred_proba = knn.predict_proba(X_train[features3])

# Print scoring
print(classification_report(y_train, y_pred))

In [ ]:
# features 3 performed better than feature 1 & 2


# Predict outcomes
y_pred = knn.predict(X_validate[features3])
# Probability estimates
y_pred_proba = knn.predict_proba(X_validate[features3])

# Print scoring
print(classification_report(y_validate, y_pred))

# KNN Feature 4

In [ ]:
# Create knn model
knn = KNeighborsClassifier(n_neighbors = 4)
# Fit model to predictors and target
knn = knn.fit(X_train[features4], y_train)
# Predict outcomes
y_pred = knn.predict(X_train[features4])
# Probability estimates
y_pred_proba = knn.predict_proba(X_train[features4])

# Print scoring
print(classification_report(y_train, y_pred))

In [ ]:
# features 3 & 4 performed nearly identical


# Predict outcomes
y_pred = knn.predict(X_validate[features4])
# Probability estimates
y_pred_proba = knn.predict_proba(X_validate[features4])

# Print scoring
print(classification_report(y_validate, y_pred))

# KNN F5

In [ ]:
# Create knn model
knn = KNeighborsClassifier(n_neighbors = 3)
# Fit model to predictors and target
knn = knn.fit(X_train[features5], y_train)
# Predict outcomes
y_pred = knn.predict(X_train[features5])
# Probability estimates
y_pred_proba = knn.predict_proba(X_train[features5])

# Print scoring
print(classification_report(y_train, y_pred))

In [ ]:


# Predict outcomes
y_pred = knn.predict(X_validate[features5])
# Probability estimates
y_pred_proba = knn.predict_proba(X_validate[features5])

# Print scoring
print(classification_report(y_validate, y_pred))

# KNN F6

In [ ]:
# Create knn model
knn = KNeighborsClassifier(n_neighbors = 3)
# Fit model to predictors and target
knn = knn.fit(X_train[features6], y_train)
# Predict outcomes
y_pred = knn.predict(X_train[features6])
# Probability estimates
y_pred_proba = knn.predict_proba(X_train[features6])

# Print scoring
print(classification_report(y_train, y_pred))

In [ ]:

# Predict outcomes
y_pred = knn.predict(X_validate[features6])
# Probability estimates
y_pred_proba = knn.predict_proba(X_validate[features6])

# Print scoring
print(classification_report(y_validate, y_pred))

# Decision Tree Exploration

In [ ]:
### 

# Make the model
tree1 = DecisionTreeClassifier(max_depth=5, random_state=123)

# Fit the model (on train and only train)
tree1 = tree1.fit(X_train[features3], y_train)

# Use the model
# We'll evaluate the model's performance on train, first
y_predictions = tree1.predict(X_train[features3])

print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(tree1.score(X_train[features3], y_train)))

In [ ]:
# confusion matrix - actual on left, predicted on top
pd.DataFrame(confusion_matrix(y_train, y_predictions))

In [ ]:
pd.crosstab(y_train, y_predictions)

In [ ]:
print(classification_report(y_train, y_predictions))

In [ ]:
# Produce the classification report on the actual y values and this model's predicted y values
report = classification_report(y_train, y_predictions, output_dict=True)
print("Tree of 1 depth")
pd.DataFrame(report)

In [ ]:
# Make the model
tree2 = DecisionTreeClassifier(max_depth=5, random_state=123)

# Fit the model (on train and only train)
tree2 = tree2.fit(X_train[features4], y_train)

# Use the model
# We'll evaluate the model's performance on train, first
y_predictions = tree2.predict(X_train[features4])

print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(tree2.score(X_train[features4], y_train)))

# loop exploration DT

In [ ]:
for i in range(2, 21):
    # Make the model
    tree = DecisionTreeClassifier(max_depth=i, random_state=123)

    # Fit the model (on train and only train)
    tree = tree.fit(X_train[features4], y_train)

    # Use the model
    # We'll evaluate the model's performance on train, first
    y_predictions = tree.predict(X_train[features4])

    # Produce the classification report on the actual y values and this model's predicted y values
    report = classification_report(y_train, y_predictions, output_dict=True)
    print(f"Tree with max depth of {i}")
    print(pd.DataFrame(report))
    print()

In [ ]:
# Keep looping, compare in-sample to out-of-sample
metrics = []

for i in range(2, 25):
    # Make the model
    tree = DecisionTreeClassifier(max_depth=i, random_state=123)

    # Fit the model (on train and only train)
    tree = tree.fit(X_train[features2], y_train)

    # Use the model
    # We'll evaluate the model's performance on train, first
    in_sample_accuracy = tree.score(X_train[features2], y_train)
    
    out_of_sample_accuracy = tree.score(X_validate[features2], y_validate)

    output = {
        "max_depth": i,
        "train_accuracy": in_sample_accuracy,
        "validate_accuracy": out_of_sample_accuracy
    }
    
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["difference"] = df.train_accuracy - df.validate_accuracy
df

In [ ]:
df[df.difference <= 0.10].sort_values(by=['validate_accuracy', 'difference'], ascending = [False,True])

# DT using feature 5

In [ ]:
for i in range(2, 21):
    # Make the model
    tree = DecisionTreeClassifier(max_depth=i, random_state=123)

    # Fit the model (on train and only train)
    tree = tree.fit(X_train[features5], y_train)

    # Use the model
    # We'll evaluate the model's performance on train, first
    y_predictions = tree.predict(X_train[features5])

    # Produce the classification report on the actual y values and this model's predicted y values
    report = classification_report(y_train, y_predictions, output_dict=True)
    print(f"Tree with max depth of {i}")
    print(pd.DataFrame(report))
    print()

In [ ]:
# Keep looping, compare in-sample to out-of-sample
metrics = []

for i in range(2, 25):
    # Make the model
    tree = DecisionTreeClassifier(max_depth=i, random_state=123)

    # Fit the model (on train and only train)
    tree = tree.fit(X_train[features5], y_train)

    # Use the model
    # We'll evaluate the model's performance on train, first
    in_sample_accuracy = tree.score(X_train[features5], y_train)
    
    out_of_sample_accuracy = tree.score(X_validate[features5], y_validate)

    output = {
        "max_depth": i,
        "train_accuracy": in_sample_accuracy,
        "validate_accuracy": out_of_sample_accuracy
    }
    
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["difference"] = df.train_accuracy - df.validate_accuracy
df

# DT F6

In [ ]:
for i in range(2, 21):
    # Make the model
    tree = DecisionTreeClassifier(max_depth=i, random_state=123)

    # Fit the model (on train and only train)
    tree = tree.fit(X_train[features6], y_train)

    # Use the model
    # We'll evaluate the model's performance on train, first
    y_predictions = tree.predict(X_train[features6])

    # Produce the classification report on the actual y values and this model's predicted y values
    report = classification_report(y_train, y_predictions, output_dict=True)
    print(f"Tree with max depth of {i}")
    print(pd.DataFrame(report))
    print()

In [ ]:
# Keep looping, compare in-sample to out-of-sample
metrics = []

for i in range(2, 25):
    # Make the model
    tree = DecisionTreeClassifier(max_depth=i, random_state=123)

    # Fit the model (on train and only train)
    tree = tree.fit(X_train[features6], y_train)

    # Use the model
    # We'll evaluate the model's performance on train, first
    in_sample_accuracy = tree.score(X_train[features6], y_train)
    
    out_of_sample_accuracy = tree.score(X_validate[features6], y_validate)

    output = {
        "max_depth": i,
        "train_accuracy": in_sample_accuracy,
        "validate_accuracy": out_of_sample_accuracy
    }
    
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["difference"] = df.train_accuracy - df.validate_accuracy
df

# Random Forest modeling

In [ ]:
#make model
forest1 = RandomForestClassifier(max_depth=4, random_state=123)

#fit model on train
forest1.fit(X_train, y_train)

#use
#evaluate
y_predictions = forest1.predict(X_train)

#produce classification report
report = classification_report(y_train, y_predictions, output_dict=True)
print("Tree of 1 depth")
pd.DataFrame(report)

In [ ]:
#confusing matrix
pd.DataFrame(confusion_matrix(y_predictions, y_train))

In [ ]:
TN, FP, FN, TP = confusion_matrix(y_train,y_predictions).ravel()
ALL = TP + TN + FP + FN

TP, TN, FP, FN

In [ ]:
accuracy = (TP + TN)/ALL
print(f"Accuracy: {accuracy}")

true_positive_rate = TP/(TP+FN)
print(f"True Positive Rate: {true_positive_rate}")

false_positive_rate = FP/(FP+TN)
print(f"False Positive Rate: {false_positive_rate}")

true_negative_rate = TN/(TN+FP)
print(f"True Negative Rate: {true_negative_rate}")

false_negative_rate = FN/(FN+TP)
print(f"False Negative Rate: {false_negative_rate}")

precision = TP/(TP+FP)
print(f"Precision: {precision}")

recall = TP/(TP+FN)
print(f"Recall: {recall}")

f1_score = 2*(precision*recall)/(precision+recall)
print(f"F1 Score: {f1_score}")

support_pos = TP + FN
print(f"Support (0): {support_pos}")

support_neg = FP + TN
print(f"Support (1): {support_neg}")

# RF F1

In [ ]:
#Loop
for i in range(2, 11):
    #make model
    forest = RandomForestClassifier(max_depth=i, min_samples_leaf=7, random_state=123)

    #fit model on train
    forest = forest.fit(X_train, y_train)

    #use
    #evaluate
    y_predictions = forest.predict(X_train)

    #produce classification report
    report = classification_report(y_train, y_predictions, output_dict=True)
    print(f'Train with max depth of {i}. \n') 
    print(pd.DataFrame(report))
    print()

In [ ]:
metrics = []

for i in range(2, 25):
    #make model
    forest = RandomForestClassifier(max_depth=i, random_state=123)

    #fit model on train
    forest = forest.fit(X_train, y_train)

    #use
    #evaluate
    in_sample_accuracy = forest.score(X_train,y_train)
    
    out_sample_accuracy = forest.score(X_validate, y_validate)
    
    output = {
        'max_depth': i,
        'train_accuracy': in_sample_accuracy,
        'validate_accuracy': out_sample_accuracy
    }
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["difference"] = df.train_accuracy - df.validate_accuracy
df

In [ ]:
df[df.difference <= 0.05].sort_values(by=['validate_accuracy', 'difference'], ascending = [False,True])

In [ ]:
metrics = []

for i in range(2, 25):
    #make model
    forest = RandomForestClassifier(max_depth=i, random_state=123)

    #fit model on train
    forest = forest.fit(X_train, y_train)

    #use
    #evaluate
    in_sample_accuracy = forest.score(X_train,y_train)
    
    out_sample_accuracy = forest.score(X_validate, y_validate)
    
    output = {
        'max_depth': i,
        'train_accuracy': in_sample_accuracy,
        'validate_accuracy': out_sample_accuracy
    }
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["difference"] = df.train_accuracy - df.validate_accuracy
df

# Random forrest [all features] max depth 4 preferredd model currently


# RF using feature 4

In [ ]:
#make model
forest2 = RandomForestClassifier(max_depth=4, random_state=123)

#fit model on train
forest2.fit(X_train[features5], y_train)

#use
#evaluate
y_predictions = forest2.predict(X_train[features5])

#produce classification report
report = classification_report(y_train, y_predictions, output_dict=True)
print("Tree of 4 depth, features 4")
pd.DataFrame(report)

In [ ]:
for i in range(2, 25):
    #make model
    forest = RandomForestClassifier(max_depth=i, random_state=123)

    #fit model on train
    forest = forest.fit(X_train[features5], y_train)

    #use
    #evaluate
    in_sample_accuracy = forest.score(X_train[features5],y_train)
    
    out_sample_accuracy = forest.score(X_validate[features5], y_validate)
    
    output = {
        'max_depth': i,
        'train_accuracy': in_sample_accuracy,
        'validate_accuracy': out_sample_accuracy
    }
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["difference"] = df.train_accuracy - df.validate_accuracy
df

In [ ]:
df[df.difference <= 0.05].sort_values(by=['validate_accuracy', 'difference'], ascending = [False,True])

# RF F6

In [ ]:
#make model
forest2 = RandomForestClassifier(max_depth=4, random_state=123)

#fit model on train
forest2.fit(X_train[features6], y_train)

#use
#evaluate
y_predictions = forest2.predict(X_train[features6])

#produce classification report
report = classification_report(y_train, y_predictions, output_dict=True)
print("Tree of 4 depth, features 4")
pd.DataFrame(report)

In [ ]:
for i in range(2, 25):
    #make model
    forest = RandomForestClassifier(max_depth=i, random_state=123)

    #fit model on train
    forest = forest.fit(X_train[features6], y_train)

    #use
    #evaluate
    in_sample_accuracy = forest.score(X_train[features6],y_train)
    
    out_sample_accuracy = forest.score(X_validate[features6], y_validate)
    
    output = {
        'max_depth': i,
        'train_accuracy': in_sample_accuracy,
        'validate_accuracy': out_sample_accuracy
    }
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["difference"] = df.train_accuracy - df.validate_accuracy
df

# Logistic Regression

In [8]:
#create LOG regression
logit = LogisticRegression(penalty='l1',solver='liblinear', C=1, random_state=123)

#Specify features


# FIT model with specified features
logit.fit(X_train, y_train)

# predict on same subset
y_pred = logit.predict(X_train)

print("Baseline is", round(baseline_accuracy, 2))
# print("Logistic Regression using features")
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train, y_train)))

Baseline is 0.73
Accuracy of Logistic Regression classifier on training set: 0.80


In [9]:
# predict on same subset
y_pred = logit.predict(X_validate)

print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.88      0.87      1239
           1       0.64      0.56      0.60       449

    accuracy                           0.80      1688
   macro avg       0.74      0.72      0.73      1688
weighted avg       0.79      0.80      0.79      1688



In [ ]:
features = ['monthly_charges', 'internet_service_type_Fiber optic', 'internet_service_type_None', 'senior_citizen']
features2 = ['tenure','monthly_charges', 'partner_encoded','senior_citizen']
features3 = ['tenure','monthly_charges', 'partner_encoded','total_charges','senior_citizen']
features4 = ['tenure','monthly_charges', 'partner_encoded']
features5 = ['internet_service_type_Fiber optic', 'internet_service_type_None','tenure','monthly_charges']
features6 = ['tenure','contract_type_One year','contract_type_Two year','internet_service_type_Fiber optic','internet_service_type_None','monthly_charges']

In [ ]:
#create LOG regression
logit = LogisticRegression(penalty='l1',solver='liblinear', C=1, random_state=123)

#Specify features


# FIT model with specified features
logit.fit(X_train[features], y_train)

# predict on same subset
y_pred = logit.predict(X_train[features])

print("Baseline is", round(baseline_accuracy, 2))
# print("Logistic Regression using features")
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train[features], y_train)))

In [ ]:
# predict on same subset
y_pred = logit.predict(X_validate)

print(classification_report(y_validate, y_pred))

In [10]:
#create LOG regression
logit = LogisticRegression(C=0.01, random_state=70)


# FIT model with specified features
logit = logit.fit(X_train, y_train)

# predict on same subset
y_pred = logit.predict(X_train)

print("Baseline is", round(baseline_accuracy, 2))
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train, y_train)))

Baseline is 0.73
Accuracy of Logistic Regression classifier on training set: 0.80


In [11]:
# predict on same subset
y_pred = logit.predict(X_validate)

print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1239
           1       0.65      0.53      0.58       449

    accuracy                           0.80      1688
   macro avg       0.75      0.71      0.73      1688
weighted avg       0.79      0.80      0.79      1688



In [12]:
# predict on test
y_pred = logit.predict(X_test)

# probability estimates
y_proba = logit.predict_proba(X_test)

# evaulate on validate
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.88      1033
           1       0.68      0.56      0.62       374

    accuracy                           0.81      1407
   macro avg       0.76      0.73      0.75      1407
weighted avg       0.80      0.81      0.81      1407



In [13]:
df1 = get_telco_data()
df2 = pd.DataFrame(y_pred)
df3 = pd.DataFrame(y_proba)
df4 = pd.merge(df2, df3, left_index=True, right_index=True)
df4.rename(columns={'0_x':'churn_prediction', '0_y':'probability_churn_false',1:'probability_churn_true'}, inplace = True)


In [ ]:
df1.head()

In [14]:
export = pd.merge(df1,df4, left_index = True, right_index=True)
export.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'gender','senior_citizen','partner','dependents','tenure','phone_service','paperless_billing','monthly_charges','total_charges','churn','contract_type','internet_service_type','payment_type',
                    'multiple_lines','online_security','online_backup','device_protection','streaming_tv','streaming_movies','tech_support'],inplace = True)
export.to_csv('predictions.csv')

In [18]:
df_pred.head()

,Unnamed: 0,customer_id,churn_prediction,probability_churn_false,probability_churn_true
0,0,0002-ORFBO,0,0.966179,0.033821
1,1,0003-MKNFE,0,0.974090,0.025910
2,2,0004-TLHLJ,1,0.420788,0.579212
3,3,0011-IGKFF,0,0.617416,0.382584
4,4,0013-EXCHZ,0,0.897406,0.102594


In [15]:
df_pred = pd.read_csv('predictions.csv')

In [20]:
df_pred = df_pred.drop(columns=['Unnamed: 0'])

In [22]:
df_pred.head()

,customer_id,churn_prediction,probability_churn_false,probability_churn_true
0,0002-ORFBO,0,0.966179,0.033821
1,0003-MKNFE,0,0.974090,0.025910
2,0004-TLHLJ,1,0.420788,0.579212
3,0011-IGKFF,0,0.617416,0.382584
4,0013-EXCHZ,0,0.897406,0.102594


In [23]:
df_pred.to_csv('predictions.csv')